# The  Algebra Of Space (G3)

In this notebook, we give a more detailed look at how to use `clifford`. 

##  Setup

First, we import clifford as `cf`, set display precision, and  instantiate a G3 algebra using `Cl()`

In [224]:
import clifford as cf
cf.pretty(precision=3)    # sets display precision and pretty-fies output
layout, blades = cf.Cl(3, firstIdx=1) # creates a 3-dimensional clifford algebra

`Cl()` is a helper function which creates the algebra and returns a `layout` and `blades`. The `layout` holds information and functions related this instance of `G3`,  and the `blades` is a dictionary which contains the basis blades, indexed by their string representations,

In [225]:
blades 

{'e1': (1.0^e1),
 'e12': (1.0^e12),
 'e123': (1.0^e123),
 'e13': (1.0^e13),
 'e2': (1.0^e2),
 'e23': (1.0^e23),
 'e3': (1.0^e3)}

You may wish to explicitly assign the blades to variables like so, 

In [226]:
e1 = blades['e1']
e2 = blades['e2']
# etc ...

Or, if you're lazy and just working in an interactive session you can use `locals()` to update your namespace with all of the blades.

In [227]:
locals().update(blades)

Now, all the blades have been defined in the local namespace

In [228]:
e3, e123

((1.0^e3), (1.0^e123))

##  Basics

###  Products

The basic products   are available

In [229]:
e1*e2 # geometric product

(1.0^e12)

In [230]:
e1|e2 # inner product 

0

In [231]:
e1^e2 # outer product

(1.0^e12)

In [232]:
e1^e2^e3 # even more outer products

(1.0^e123)

###  Defects in Precidence

But note that Python's operator precidence makes the outer product evaluate after addition. This requires the use of parenthesis when using outer products. For example

In [233]:
e1^e2+e2^e3 # fail

(2.0^e123)

In [234]:
(e1^e2) + (e2^e3) # correct

(1.0^e12) + (1.0^e23)

Also the inner product of a scalar and a Multivector is  0, 

In [235]:
4|e1

0

So for scalars, use the outer product or geometric product instead

In [236]:
4*e1

(4.0^e1)

### Multivectors

Multivectors can be defined in terms of the basis blades. For example you can construct a rotor as a sum of a scalar and bivector, like so 

In [237]:
theta = pi/4
R = cos(theta) - sin(theta)*e23
R

0.707 - (0.707^e23)

You can also mix grades without any reason


In [238]:
A = 1 + 2*e1 + 3*e12 + 4*e123
A

1.0 + (2.0^e1) + (3.0^e12) + (4.0^e123)

### Reversion

The reversion operator is accomplished with the tilde `~` in front of the Multivector on which it acts

In [239]:
~A

1.0 + (2.0^e1) - (3.0^e12) - (4.0^e123)

### Grade Projection

Taking a projection onto a specific grade $n$  of a Multivector is usually written 

$$\langle A \rangle _n$$

can be done by using soft brackets, like so

In [259]:
A(0) # get grade-0 elements of R

1.0

In [260]:
A(1) # get grade-1 elements of R

(2.0^e1)

In [261]:
A(2)  #  you get it

(3.0^e12)

### Magnitude

Using the reversion and grade projection operators, we can define the magnitude of $A$

$$|A|^2 = \langle A\tilde{A}\rangle$$

In [246]:
(~A*A)(0)

30.0

This is  done in the `abs()` operator

In [247]:
abs(A)**2

30.0

### Inverse

The inverse of a Multivector is defined as $A^{-1}A=1$

In [248]:
A.inv() *A

1.0

In [249]:
A.inv()

0.134 + (0.122^e1) - (0.146^e3) + (0.183^e12) + (0.098^e23) - (0.293^e123)

## Applications

### Reflections

Reflections are pretty simple, 

$$ a \rightarrow -nan$$

In [250]:
a = e1+e2+e3    # the vector
n = e1          # the reflector
-n*a*n          # reflect `a` in hyperplane normal to `n`

-(1.0^e1) + (1.0^e2) + (1.0^e3)

Because we have the `inv()` available, we can reflect in un-normalized vectors using, 
$$ a \rightarrow -nan^{-1}$$

In [251]:
a = e1+e2+e3    # the vector
n = 3*e1          # the reflector
-n*a*n.inv()

-(1.0^e1) + (1.0^e2) + (1.0^e3)

### Rotations

A vector can be rotated using the formula
$$ a \rightarrow Ra\tilde{R}$$

Where $R$ is a rotor. A rotor can be defined by multiple reflections, or by a plane and an angle, the latter being more common for applications. For example

In [252]:
from scipy.constants import pi

R = e**(-pi/4*e12) # enacts rotation by pi/2 
R

0.707 - (0.707^e12)

In [253]:
R*e1*~R    # rotate e1 by pi/2 in the e12-plane

(1.0^e2)

Maybe we want to define a function which can return rotor of some angle $\theta$ in the $e_{12}$-plane,

$$ R_{12} = e^{-\frac{\theta}{2}e_{12}} $$

In [254]:
R12 = lambda theta: e**(-theta/2*e12)
R12(pi/2)

0.707 - (0.707^e12)

And use it like this

In [255]:
a = e1+e2+e3
R = R12(pi/2)
R*a*~R


-(1.0^e1) + (1.0^e2) + (1.0^e3)

More complicated rotations can be built up quickly from simpler ones, for example


\begin{align}
R &= R_{13}R_{23}R_{12} \\
 &=  e^{-\frac{\psi}{2}e_{13}}e^{-\frac{\theta}{2}e_{23}}e^{-\frac{\phi}{2}e_{12}}
\end{align}

In [256]:
R12 = lambda theta: e**(-theta/2*e12) # theta is just a dummy variable, so it cand be re-used
R23 = lambda theta: e**(-theta/2*e23)
R13 = lambda theta: e**(-theta/2*e13)

def Rot3D(phi,theta, psi,):
    '''
    Returns rotor for three successive rotations in the e12,e23,and e13-planes,
    in that order
    '''
    return R13(psi)*R23(theta)*R12(phi)

Rot3D(pi/2, pi/3,pi/4)

0.43 - (0.701^e12) - (0.561^e13) - (0.092^e23)